In [0]:
%run "../Configuration"

In [0]:
# Define the metadata widget:
dbutils.widgets.text("entityName", "")
dbutils.widgets.text("entityIngestionColumns", "")
dbutils.widgets.text("sourceSystem", "")

# Save parameters to variables:
entityName =       dbutils.widgets.get('entityName')
sourceSystemName = dbutils.widgets.get('sourceSystem')
columns =          json.loads(dbutils.widgets.get('entityIngestionColumns'))
# Query sourceEntity table to find location of raw data:
raw_path =         entity_bronze(entityName)

In [0]:
# Ignore if date:
if entityName == 'date':
    # Currently no date table.  Exit notebook:
    dbutils.notebook.exit('None')

In [0]:
# Create dataframe:
df = spark.read.format('parquet').load(BRONZE + raw_path)

In [0]:
if entityName == 'currency':
    tv_currency = df.createOrReplaceTempView('tv_currency')
    query = f"""
    SELECT CAST(currency_id AS {columns[0]['dataType']}) AS {columns[0]['columnName']}
    ,      {columns[1]['dataType']}(currency_code) AS {columns[1]['columnName']}
    ,      CASE 
                WHEN currency_code = 'USD' THEN 'United States Dollar'
                WHEN currency_code = 'EUR' THEN 'Euro'
                WHEN currency_code = 'JPY' THEN 'Japanese Yen'
                WHEN currency_code = 'GBP' THEN 'British Pound Sterling'
                WHEN currency_code = 'AUD' THEN 'Australian Dollar'
                WHEN currency_code = 'CAD' THEN 'Canadian Dollar'
                WHEN currency_code = 'CHF' THEN 'Swiss Franc'
                WHEN currency_code = 'CNY' THEN 'Chinese Yuan'
                WHEN currency_code = 'NZD' THEN 'New Zealand Dollar'
                ELSE 'Unknown Currency'
           END AS {columns[2]['columnName']}   
    FROM tv_currency
    """
    
    # Execute Query:
    df = spark.sql(query)

In [0]:
# Cast data types:
df = cast_data_types_ingestion(df)

# Drop unwanted columns:
df = drop_unwanted_columns_ingestion(df)

# Drop duplicate values:
df = drop_duplicates_ingestion(df)
    

In [0]:
# Check data exists:
if df.count() > 0:
    # Set location for delta table:
    location = f"/{sourceSystemName}/{entityName}/"
    
    # Write to delta table:
    df.write.mode('overwrite').format('delta').option('header', 'true').save(SILVER + location)

    # Output location:
    dbutils.notebook.exit(location)
else:
    dbutils.notebook.exit(-1)
